# LangChain: Models, Prompts and Output Parsers

- Direct API calls to OpenAI
- API calls through LangChain:
    - Prompts
    - Models
    - Output parsers
---

## Setup

In [5]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

## Chat API: OpenAI
Direct API calls to OpenAI

In [6]:
def get_completion(
    user_prompt,
    system_prompt="Help users with their questions.",
    deployment_id="gpt4",
    max_tokens=400,
):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        temperature=0,
        max_tokens=max_tokens,
    )
    # print(response)
    return response.choices[0].message["content"]

In [7]:
system_prompt = "Help users with their questions."
user_prompt = "What's 1+1?"
get_completion(user_prompt, system_prompt)

'1+1 equals 2.'

In [8]:
customer_email = """\
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!\
"""

style = """American English \
in a calm and respectful tone\
"""

prompt = f"""Translate the text \
that is delimited by triple backticks \
into a style that is {style}.
text: ```{customer_email}```\
"""

get_completion(prompt)

"```\nI am quite frustrated that my blender lid came off and splattered my kitchen walls with smoothie! To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help right now, my friend!\n```"

## Chat API: LangChain
Abstracts away the OpenAI API and provides a simple interface to chat with the AI.

In [9]:
from langchain.chat_models import AzureChatOpenAI

chat = AzureChatOpenAI(
    deployment_name="gpt4",
    openai_api_version="2023-03-15-preview",
    temperature=0,
)

### Prompt Template

In [10]:
template_string = """Translate the string that is delimited by single quotes into a style \
that is {style}. \
text: '{text}'\
"""

In [11]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [12]:
customer_style = "American English in a clam and respectful tone"
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen! I need yer help \
right now, matey!
"""

prompt = prompt_template.format_messages(style=customer_style, text=customer_email)
prompt

[HumanMessage(content="Translate the string that is delimited by single quotes into a style that is American English in a clam and respectful tone. text: '\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen! I need yer help right now, matey!\n'", additional_kwargs={}, example=False)]

In [13]:
response = chat(prompt)
response.content

'I am quite frustrated that my blender lid came off and splattered my kitchen walls with smoothie! To make matters worse, the warranty does not cover the cost of cleaning up my kitchen! I need your help right now, my friend!'

### Output Parsers
Allows for the output to be parsed and modified before being returned to the user.

#### The default type is str for the LLM output.

In [14]:
{"gift": False, "delivery_days": 5, "price_value": "pretty affordable!"}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [15]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [16]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [17]:
messages = prompt_template.format_messages(text=customer_review)
response = chat(messages)
print(response.content)

{
  "gift": True,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [18]:
response.content.get("gift")

AttributeError: 'str' object has no attribute 'get'

#### Parse the LLM output string into a Python dictionary

In [19]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [20]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.",
)
delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.",
)
price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.",
)

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [21]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.\n\t"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.\n\t"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.\n}\n```'

In [22]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)
messages = prompt.format_messages(
    text=customer_review, format_instructions=format_instructions
)

In [23]:
response = chat(messages)
output_dict = output_parser.parse(response.content)
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [24]:
output_dict.get("gift")

'True'